In [45]:
import torch
import numpy as np

In [46]:

#### Hyper parameters ####

num_arms = 5
num_agents = 5
max_epochs = 1000
epsilon = 5
influence_weight = 0.5
device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

##########################


### Initialization ###

arm_performance = torch.rand(num_arms, 2, device=device)
history = torch.zeros(1, num_agents, num_arms, device=device)
adjacency_matrix = torch.rand(num_agents, num_agents, device=device)
######################

best_arm = torch.argmax(arm_performance[:, 0])
print(f"Best arm is {best_arm+1} ")

Best arm is 4 


In [47]:
performance_estimate = history.sum(dim=0) / (history.sum(dim=0).sum(dim=1).unsqueeze(1) + 1)
performance_estimate

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')

In [48]:
averaging_runs = 100

averaged_history = torch.zeros( num_agents, num_arms, device=device, requires_grad=False)

adjacency_matrix = torch.clamp(adjacency_matrix, min=0, max=1)


for run in range(0, averaging_runs):     

    history = torch.zeros(1, num_agents, num_arms, device=device)   
    for epoch in range(1, max_epochs):

            performance_estimate = history.sum(dim=0) / (history.sum(dim=0).sum(dim=1).unsqueeze(1) + 1)
            choice = torch.argmax(performance_estimate + torch.sqrt(np.log(epoch)/(1+history.bool().sum(dim=0).int())) + influence_weight * adjacency_matrix.softmax(dim=0) @ performance_estimate, dim=1)
            
            rewards = torch.normal(mean=arm_performance[choice, 0], std=torch.abs(arm_performance[choice, 1]))


            current_history  = torch.zeros(num_agents, num_arms, device=device)

            for i in range(num_agents):
                for j in range(num_arms):
                    if j == choice[i]:
                        current_history[i, j] = rewards[i]


            history = torch.cat((history, current_history.unsqueeze(0)), dim=0)



    history = history.bool().sum(dim=0).float()
    averaged_history += history

averaged_history /= averaging_runs


print(averaged_history)

        




In [ ]:
history_true = history
average_history_true = averaged_history
adjacency_matrix_true = adjacency_matrix

In [ ]:
train_epochs = 1000


adjacency_matrix = torch.rand(num_agents, num_agents, device=device, requires_grad=True)

averaging_runs = 100

optimizer = torch.optim.Adam([adjacency_matrix], lr=10)

for train_epoch in range(train_epochs):

    

    adjacency_matrix = torch.clamp(adjacency_matrix, min=0, max=1)

    averaged_history = torch.zeros(num_agents, num_arms, device=device, requires_grad=True)

    for run in range(0, averaging_runs):

        history = torch.zeros(2, num_agents, num_arms, device=device, requires_grad=True)
    
        for epoch in range(1, max_epochs):
            performance_estimate = history.sum(dim=0) / (history.sum(dim=0).sum(dim=1).unsqueeze(1) + 1) + epsilon * torch.sqrt(np.log(epoch)/(1+history.bool().sum(dim=0).int()))
            choice = torch.argmax(performance_estimate + influence_weight * adjacency_matrix @ performance_estimate, dim=1)
            
            rewards = torch.normal(mean=arm_performance[choice, 0], std=torch.abs(arm_performance[choice, 1]))


            current_history  = torch.zeros(num_agents, num_arms, device=device)

            for i in range(num_agents):
                for j in range(num_arms):
                    if j == choice[i]:
                        current_history[i, j] = rewards[i]


            history = torch.cat((history, current_history.unsqueeze(0)), dim=0)

        history = history.bool().sum(dim=0).float() / history.bool().sum(dim=0).float().max()

        averaged_history = averaged_history + history
        
    averaged_history = averaged_history / averaging_runs
    
  
 

    #Frobenius norm
    loss = torch.norm(averaged_history - average_history_true, p='fro')


    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    print(f"Epoch {train_epoch} Loss: {loss}")



KeyboardInterrupt: 

In [ ]:
adjacency_matrix = torch.rand(num_agents, num_agents, device=device, requires_grad=True)
adjacency_matrix

tensor([[0.6888, 0.2940, 0.2669, 0.7875, 0.2519],
        [0.6291, 0.5302, 0.3250, 0.6518, 0.6863],
        [0.4272, 0.0538, 0.5969, 0.1319, 0.4401],
        [0.1027, 0.8306, 0.1849, 0.3191, 0.6685],
        [0.6106, 0.2331, 0.1795, 0.8182, 0.6294]], device='cuda:0',
       requires_grad=True)

In [ ]:
# Adjacency matrix / max element 
adjacency_matrix = adjacency_matrix / adjacency_matrix.max()
adjacency_matrix

tensor([[0.8293, 0.3539, 0.3214, 0.9480, 0.3032],
        [0.7574, 0.6383, 0.3913, 0.7847, 0.8262],
        [0.5143, 0.0648, 0.7186, 0.1588, 0.5298],
        [0.1237, 1.0000, 0.2227, 0.3842, 0.8048],
        [0.7351, 0.2806, 0.2161, 0.9850, 0.7577]], device='cuda:0',
       grad_fn=<DivBackward0>)

In [ ]:
torch.sigmoid(2 * 0.9458 - 1)

TypeError: sigmoid(): argument 'input' (position 1) must be Tensor, not float

In [ ]:
history = torch.zeros(2, num_agents, num_arms, device=device, requires_grad=True)
history

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], device='cuda:0', requires_grad=True)

In [ ]:
performance_estimate = history.sum(dim=0) / (history.sum(dim=0).sum(dim=1).unsqueeze(1) + 1) + epsilon * torch.sqrt(np.log(epoch)/(1+history.bool().sum(dim=0).int()))
performance_estimate

tensor([[1.6651, 1.6651, 1.6651, 1.6651, 1.6651],
        [1.6651, 1.6651, 1.6651, 1.6651, 1.6651]], device='cuda:0',
       grad_fn=<AddBackward0>)